<a href="https://colab.research.google.com/github/tsugumasa320/VAE_Study/blob/main/pytorch_lightning_AE4Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pystyle.info/pytorch-lightning-auto-encoder-train-mnist/
参考

1. 必要なモジュールを import する

#課題

    ・データセットを自作のものに置き換え
    ・データの入力方法を考える
    ・モデルをVAEに改造
    ・オプティマイザー再度確認
    ・ラベルを読み込む
    ・ラベルを使って操作をする

In [3]:
! pip install --quiet pytorch_lightning

     |████████████████████████████████| 585 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 41.5 MB/s 
     |████████████████████████████████| 140 kB 31.4 MB/s 
     |████████████████████████████████| 419 kB 50.5 MB/s 
     |████████████████████████████████| 1.1 MB 34.4 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 271 kB 50.5 MB/s 
     |████████████████████████████████| 144 kB 50.8 MB/s 


In [4]:
from pathlib import Path
from typing import Optional, Union, Tuple # 型ヒント
from torch.utils.data import DataLoader, Dataset #追加

import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision # 不要になるかも
import torchvision.transforms as transforms # 同上
import torchaudio
import gdown

In [7]:
# @title Prepare data and utility functions. {display-mode: "form"}
# @markdown
# @markdown You do not need to look into this cell.
# @markdown Just execute once and you are good to go.

# -------------------------------------------------------------------------------
# Preparation of data and helper functions.
# -------------------------------------------------------------------------------
import multiprocessing
import os

import matplotlib.pyplot as plt
from IPython.display import Audio, display


_SAMPLE_DIR = "_assets"
DATASET_PATH = os.path.join(_SAMPLE_DIR, "dataset")
os.makedirs(DATASET_PATH, exist_ok=True)


def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
        if xlim:
            axes[c].set_xlim(xlim)
    figure.suptitle(title)
    plt.show(block=False)


def play_audio(waveform, sample_rate):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    if num_channels == 1:
        display(Audio(waveform[0], rate=sample_rate))
    elif num_channels == 2:
        display(Audio((waveform[0], waveform[1]), rate=sample_rate))
    else:
        raise ValueError("Waveform with more than 2 channels are not supported.")

2. LightningDataModule を作成する

In [13]:
gdown.download('https://drive.google.com/uc?id=1keOTwrdNqkFWFXQ0kQ5x8obp-prSolRj', 'nsynth-subset.zip' , quiet=True) #全部で２２個のファイル
!unzip -q nsynth-subset

replace nsynth-subset/vocal_acoustic_000-056-025.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [5]:
class NsynthSubset(Dataset):
    """Create a Dataset for Nsynth subset.
        Args:
        root (str or Path): Path to the directory where the dataset is found.
    """

def __init__(self, root: Union[str, Path]) -> None: # 最後の　, 不要だよね？ 

    self._parse_filesystem(root, url, folder_in_archive, download)

def __getitem__(self, n: int) -> Tuple[Tensor, int, List[int]]:

    """Load the n-th sample from the dataset.
    Args:
        n (int): The index of the sample to be loaded
    Returns:
        (Tensor, int, List[int]): ``(waveform, sample_rate, labels)``
    """

def __len__(self) -> int
    return len(self._walker)


SyntaxError: ignored

In [8]:
# LightningDataModuleはDataLoader(場合によってはDatasetも)となるクラス

    class MyDataModule(pl.LightningDataModule):
    
    def __init__(self, batch_size, data_dir="./"):
        super().__init__() #親クラスのinit

        dataset = torchaudio.datasets.YESNO( # YESNOのDL
            root=data_dir, download=True)

        train_dataset, val_dataset = torch.utils.data.random_split( #trainとvalを分ける
            dataset, [55, 5]
        )
        test_dataset = torchaudio.datasets.YESNO( # YESNOのDL設定とテンソル化
            root=data_dir, download=True)

        for i in range(55):
            self.train_dataset = train_dataset[i][0][:,:32000] #4秒で切り捨て(8000[sr]*4[s]). これで大丈夫か？
        for j in range(5):
            self.val_dataset = val_dataset[j][0][:,:32000]
            self.test_dataset = test_dataset[j][0][:,:32000]

        self.batch_size = batch_size
        self.data_dir = data_dir

    def train_dataloader(self): # Train用DataLoaderの設定
        return torch.utils.data.DataLoader(
            self.train_dataset, batch_size=self.batch_size, num_workers=8
        )

    def val_dataloader(self): # val用DataLoaderの設定
        return torch.utils.data.DataLoader(
            self.val_dataset, batch_size=self.batch_size, num_workers=8
        )

    def test_dataloader(self): # Test用DataLoaderの設定
        return torch.utils.data.DataLoader(
            self.test_dataset, batch_size=self.batch_size, num_workers=8
        )

    def predict_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_dataset, batch_size=self.batch_size, num_workers=8
        )
a = MyDataModule(batch_size=32)
b=a.train_dataset.shape
print(b)
play_audio(b, 8000)

#for data in a.train_dataloader():
#    print(data)
#    break
#print(data)

torch.Size([1, 32000])


AttributeError: ignored

3. LightningModule を作成する

LightningModule にモデルや各処理を定義します。 モデルの構造ですが、

1.   784次元の入力層

2.   64次元の全結合層

3.   ReLU

4.   32次元の全結合層

5.   64次元の全結合層

6.   ReLU

7.   784次元の全結合層

となっており、入力画像と同じ画像を出力することが学習の目標なの

で、入力画像と出力画像の差異を mse_loss() で計算します。



In [18]:
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, hidden_dim=1000): #Define computations here
        super().__init__()
        self.encoder = nn.Sequential( # encoder設定(784,64,32)
            nn.Linear(32000, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, 32)
        )
        self.decoder = nn.Sequential(# decoder設定(32,64,784)
            nn.Linear(32, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, 32000)
        )

    def forward(self, x): # Use for inference only (separate from training_step)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

    def training_step(self, batch, batch_idx): # the complete training loop
        return self._common_step(batch, batch_idx, "train")

    def validation_step(self, batch, batch_idx): # the complete validation loop
        self._common_step(batch, batch_idx, "val")

    def test_step(self, batch, batch_idx): # the complete test loop
        self._common_step(batch, batch_idx, "test")

    def predict_step(self, batch, batch_idx, dataloader_idx=None): # the complete prediction loop
        x, _ = batch
        return self(x)

    def configure_optimizers(self): #Optimizerと学習率(lr)設定
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def _prepare_batch(self, batch): #batch準備
        x = batch #x, _ = batch
        return x.view(x.size(0), -1)

    def _common_step(self, batch, batch_idx, stage): #ロス関数定義
        x = self._prepare_batch(batch)
        loss = F.mse_loss(x, self(x)) #
        self.log(f"{stage}_loss", loss, on_step=True)
        return loss

4.Datasetを用意する

　4-1.torchaudioからダウンロード

　4-2.

4. 学習途中の生成結果を画像化するコールバック関数を作成する

生成する枚数を torch.utils.data.DataLoader() のバッチサイズに指定し、検証用のデータセットから画像を取得します。

(N, C, H, W) -> (N, C H W) に形状を変更します。

Auto Encoder に入力し、画像を生成します。

(N, C H W) -> (N, C, H, W) に形状を変更します。

torchvision.utils.make_grid() で画像を結合します。

torchvision.utils.save_image() で画像を保存します。

In [19]:
class MyPrintingCallback(pl.callbacks.Callback):
    def __init__(
        self, output_dir: str = "./generated", num_samples: int = 64, nrow: int = 8
    ):
        super().__init__()
        self.num_samples = num_samples

    def on_train_start(self, trainer, pl_module):
        print("Training is starting")

    def on_train_end(self, trainer: pl.Trainer, model: pl.LightningModule) -> None:
        # 生成する枚数をバッチサイズに指定し、検証用のデータセットから取得する。
        dataloader = torch.utils.data.DataLoader(
            trainer.datamodule.val_dataset, batch_size=self.num_samples
        )
        img_in = next(iter(dataloader))

        with torch.no_grad():
            model.eval()
            wav_out = model(img_in.to(model.device))
            model.train()

        print(wav_out)
        play_audio(wav_out, 8000)
        print("Training is ending")

5. Trainer を作成して、学習する

In [21]:
dm = MyDataModule(batch_size=32, data_dir="./")
model = LitAutoEncoder()

trainer = pl.Trainer(
    max_epochs=10, log_every_n_steps=1, gpus=None, callbacks=[MyPrintingCallback()]
)

trainer.fit(model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 32.0 M
1 | decoder | Sequential | 32.1 M
---------------------------------------
64.1 M    Trainable params
0         Non-trainable params
64.1 M    Total params
256.392   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training is starting


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

tensor([[-0.0119,  0.0242,  0.0069,  ..., -0.0050, -0.0088, -0.0195]])


Training is ending


Auto Encoder で次元削減
学習した Auto Encoder の Encoder で抽出した特徴量を使って、学習してみます。

学習した Auto Encoder の Encoder で (784,) -> (32,) に次元削減する

SVM で学習する

精度を計算する

In [ ]:
def extract(dataloader):
    X, y = [], []
    for x_batch, y_batch in dataloader:
        x_batch = x_batch.view(x_batch.size(0), -1)
        feat = model.encoder(x_batch)

        X.append(feat)
        y.append(y_batch)

    X = torch.concat(X).detach().numpy()
    y = torch.concat(y).detach().numpy()

    return X, y


# 学習データ、テストデータを Encoder で特徴量に変換する
X_train, y_train = extract(dm.train_dataloader())
X_test, y_test = extract(dm.test_dataloader())

In [ ]:
from sklearn import metrics, svm

clf = svm.SVC(kernel="linear", C=1, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

acc = metrics.accuracy_score(y_test, y_pred)
print(f"accuracy: {acc:.2%}")